In [232]:
import torch
import pyro
import pyro.distributions as dist
import pyro.contrib.gp as gp

# Define the GPR model using Pyro
pyro.set_rng_seed(10)
def gpr_pyro(x_train, y_train, x_test, noise_std, length_scale):
    n = x_train.shape[0]

    # Define the covariance function
    kernel = gp.kernels.RBF(input_dim=1, lengthscale=torch.tensor(length_scale))

    # Define the GPR model
    gp_model = gp.models.GPRegression(x_train, y_train, kernel, noise=torch.tensor(noise_std))


    #Predict on the test data
    with torch.no_grad():
        pred, _ = gp_model(x_test)

    return pred.detach().numpy()

In [233]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Define the GPR model using scikit-learn
def gpr_sklearn(x_train, y_train, x_test, noise_std, length_scale):
    kernel = RBF(length_scale=length_scale, length_scale_bounds=(1e-3, 1e3))
    gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1, optimizer=None, alpha=0.01)
    gp_model.fit(x_train, y_train)
    pred, _ = gp_model.predict(x_test, return_std=True)
    return pred

In [255]:
from sklearn.metrics import mean_absolute_error

# Generate some dummy data
np.random.seed(42)
n_samples = 10000
x = np.linspace(0, 10, n_samples)
y = np.sin(x) + np.random.normal(scale=0.1, size=n_samples)

# Split the data into training and test sets
x_train, y_train = x[:30], y[:30]
x_test, y_test = x[30:], y[30:]

# Define hyperparameters
noise_std = 0.1
length_scale = 1

# Make predictions using Pyro and scikit-learn

In [256]:

pred_pyro = gpr_pyro(torch.tensor(x_train), torch.tensor(y_train), torch.tensor(x_test), noise_std, length_scale)

In [257]:

pred_sklearn = gpr_sklearn(x_train.reshape(-1, 1), y_train, x_test.reshape(-1, 1), noise_std, length_scale)

In [258]:
# Compute MAE for Pyro and scikit-learn predictions
mae_pyro = mean_absolute_error(y_test, pred_pyro)
mae_sklearn = mean_absolute_error(y_test, pred_sklearn)

# Print the results
print("MAE for Pyro implementation: {:.4f}".format(mae_pyro))
print("MAE for scikit-learn implementation: {:.4f}".format(mae_sklearn))

MAE for Pyro implementation: 0.6241
MAE for scikit-learn implementation: 0.6687


In [246]:
# Import necessary libraries
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import Predictive

# Define input data
X = np.random.rand(10, 1)
y = np.sin(10*X)

# Define kernel and hyperparameters
kernel = RBF(length_scale=1.0,length_scale_bounds=(1e-3, 1e3))
length_scale = torch.tensor(kernel.length_scale)
noise = torch.tensor(1)

# Define Scikit-learn GPR model
sklearn_model = GaussianProcessRegressor(kernel=kernel, alpha=noise, n_restarts_optimizer=0)
sklearn_model.fit(X, y)

# Define Pyro GPR model
def model(X, y=None):
    # Define prior distributions for hyperparameters
    length_scale_prior = dist.LogNormal(torch.tensor(0.), torch.tensor(1.))
    noise_prior = dist.LogNormal(torch.tensor(0.), torch.tensor(1.))
    
    # Sample hyperparameters from prior distributions
    length_scale = pyro.sample("length_scale", length_scale_prior)
    noise = pyro.sample("noise", noise_prior)
    
    # Define kernel using sampled hyperparameters
    kernel = RBF(length_scale=length_scale.item())
    
    # Define likelihood function
    X = torch.tensor(X)
    y = pyro.sample("y", dist.MultivariateNormal(torch.zeros(X.size(0)), kernel(X)))
    
    return y

# Generate predictions using Scikit-learn GPR model
y_pred_sklearn, std_pred_sklearn = sklearn_model.predict(X, return_std=True)

# Generate predictions using Pyro GPR model
predictive = Predictive(model, num_samples=1000)
samples = predictive(X)
y_pred_pyro = samples["y"].mean(dim=0)
std_pred_pyro = samples["y"].std(dim=0)

# Calculate Mean Absolute Error (MAE) for both models
mae_sklearn = np.mean(np.abs(y_pred_sklearn - y))
mae_pyro = np.mean(np.abs(y_pred_pyro.numpy() - y))

# Print results
print("Scikit-learn GPR MAE:", mae_sklearn)
print("Pyro GPR MAE:", mae_pyro)

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.